In [1]:
import argparse
import os
import copy
import sys
import time
from datetime import datetime
import torch
import random
import numpy as np
from matplotlib import pyplot as plt
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
sys.path.append(os.path.dirname(os.path.expanduser('~/dev/pytorchSPH2/Cconv')))
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.typing import Adj, OptPairTensor, OptTensor, Size
from torch_geometric.utils.repeat import repeat
import torch
from torch import Tensor, nn
from torch.nn import Parameter

from Cconv.cutlass import cutlass

import math
import numpy as np
from typing import Any
from typing import List, Tuple, Union
from torch_sparse import SparseTensor

def uniform(size: int, value: Any):
    if isinstance(value, Tensor):
        bound = 1.0 / math.sqrt(size)
        value.data.uniform_(-bound, bound)
    else:
        for v in value.parameters() if hasattr(value, 'parameters') else []:
            uniform(size, v)
        for v in value.buffers() if hasattr(value, 'buffers') else []:
            uniform(size, v)
     
def constant(value: Any, fill_value: float):
    if isinstance(value, Tensor):
        value.data.fill_(fill_value)
    else:
        for v in value.parameters() if hasattr(value, 'parameters') else []:
            constant(v, fill_value)
        for v in value.buffers() if hasattr(value, 'buffers') else []:
            constant(v, fill_value)
def zeros(value: Any):
    constant(value, 0.)
    
basestring = (str, bytes)
def is_list_of_strings(lst):
        if lst and isinstance(lst, list):
            return all(isinstance(elem, basestring) for elem in lst)
        else:
            return False
from Cconv.cutlass import *
import scipy.optimize

In [2]:

from Cconv.plotting import *
plt.style.use('dark_background')
from tqdm.notebook import trange, tqdm

In [3]:
from Cconv.datautils import *
from tqdm import trange, tqdm

/home/winchenbach/anaconda3/envs/gns/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject
  return f(*args, **kwds)
/home/winchenbach/anaconda3/envs/gns/lib/python3.7/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters


In [4]:
basePath = '~/dev/datasets/generative2D/train/'
basePath = '/mnt/data/datasets/generative2D/train'
basePath = os.path.expanduser(basePath)

# simulationFiles = [basePath + '/' + f for f in os.listdir(basePath) if f.endswith('.zst')]
simulationFiles = [basePath + '/' + f for f in os.listdir(basePath) if f.endswith('.hdf5')]
# for i, c in enumerate(simulationFiles):
#     print(i ,c)
#     
# simulationFiles  = [simulationFiles[0]]
# simulationFiles = simulationFiles[:1]

training = []
validation = []
testing = []

for s in tqdm(simulationFiles):
    f, s, u = splitFile(s, split = False, cutoff = -4, skip = 0)
    training.append((f, (s,u)))
# for s in simulationFiles:    
#     _, train, valid, test = splitFileZSTD(s, split = True, limitRollOut = False, skip = 0, cutoff = 1800, distance = 1)
#     training.append((s,train))
#     validation.append((s,valid))
#     testing.append((s,test))
# debugPrint(training)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 24.04it/s]


In [5]:

from tqdm import tqdm


In [6]:

def processDataLoaderIter(iterations, e, rollout, ds, dataLoader, dataIter, model, optimizer, train = True, prefix = '', augmentAngle = False, augmentJitter = False, jitterAmount = 0.01):
    with record_function("prcess data loader"): 
#     pbl = gtqdms[args.gpu + args.gpus]
        losses = []
        batchIndices = []

        if train:
            model.train(True)
        else:
            model.train(False)

    #     with portalocker.Lock('README.md', flags = 0x2, timeout = None):
        t.reset(total=iterations)
        i = 0
        for b in range(iterations):
            try:
                bdata = next(dataIter)
            except:
                dataIter = iter(dataLoader)
                bdata = next(dataIter)
                
            with record_function("prcess data loader[batch]"): 
                if train:
                    optimizer.zero_grad()
                batchLosses, meanLosses, minLosses, maxLosses, stdLosses = processBatch(model, device, True, e, rollout, ds, bdata, frameDistance, augmentAngle, augmentJitter, jitterAmount)
                batchIndices.append(np.array(bdata))
                losses.append(batchLosses.detach().cpu().numpy())

                with record_function("prcess data loader[batch] - backward"): 
                    sumLosses = torch.mean(batchLosses[:,:,0]) #+ torch.mean(batchLosses[:,:,1])
                    if train:
                        sumLosses.backward()
                        optimizer.step()
                lossString = np.array2string(torch.mean(batchLosses[:,:,0],dim=0).detach().cpu().numpy(), formatter={'float_kind':lambda x: "%.4e" % x})
                batchString = str(np.array2string(np.array(bdata), formatter={'float_kind':lambda x: "%.2f" % x, 'int':lambda x:'%04d' % x}))

        #         with portalocker.Lock('README.md', flags = 0x2, timeout = None):
                t.set_description('%24s[gpu %d]: %3d [%1d] @ %1.5e: :  %s -> %.4e' %(prefix, 0, e, rollout, lr, batchString, sumLosses.detach().cpu().numpy()))
                t.update()
                if prefix == 'training':
                    te.set_description('[gpu %d] Learning: %1.4e Validation: %1.4e' %(0, np.mean(np.mean(np.vstack(losses)[:,:,0], axis = 1)), 0))
                if prefix == 'validation':
                    te.set_description('[gpu %d] Learning: %1.4e Validation: %1.4e' %(0, trainLoss, np.mean(np.mean(np.vstack(losses)[:,:,0], axis = 1))))
                te.update()
#                 i = i + 1
#                 if i > 100:
#                     break
        bIndices  = np.hstack(batchIndices)
        losses = np.vstack(losses)

        idx = np.argsort(bIndices)
        bIndices = bIndices[idx]
        losses = losses[idx]

        epochLoss = losses
        return epochLoss


In [7]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:

def constructFluidFeatures(attributes, inputData):
    fluidFeatures = torch.hstack(\
                (torch.ones(inputData['fluidArea'].shape[0]).type(torch.float32).unsqueeze(dim=1), \
                 inputData['fluidVelocity'].type(torch.float32), 
                 torch.zeros(inputData['fluidArea'].shape[0]).type(torch.float32).unsqueeze(dim=1)))
                #  inputData['fluidGravity'].type(torch.float32)))

                #  torch.ones(inputData['fluidArea'].shape[0]).type(torch.float32).unsqueeze(dim=1)))

    # fluidFeatures = torch.ones(inputData['fluidArea'].shape[0]).type(torch.float32).unsqueeze(dim=1)
    # fluidFeatures[:,0] *= 7 / np.pi * inputData['fluidArea']  / attributes['support']**2
    
    boundaryFeatures = torch.hstack((inputData['boundaryNormal'].type(torch.float32), torch.zeros(inputData['boundaryNormal'].shape[0]).type(torch.float32).unsqueeze(dim=1)))
    # boundaryFeatures = torch.ones(inputData['boundaryNormal'].shape[0]).type(torch.float32).unsqueeze(dim=1)
    # boundaryFeatures[:,0] *=  7 / np.pi * inputData['boundaryArea']  / attributes['support']**2
    
    return inputData['fluidPosition'].type(torch.float32), inputData['boundaryPosition'].type(torch.float32), fluidFeatures, boundaryFeatures


In [9]:
batch_size = 2

train_ds = datasetLoader(training)
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size = batch_size).batch_sampler

# fileName, frameIndex, maxRollout = train_ds[0]
fileName, frameIndex, maxRollout = train_ds[len(train_ds)//2]

attributes, inputData, groundTruthData = loadFrame(fileName, 1854, 1 + np.arange(8), 16)
attributes['support'] = attributes['support']# * 4.25
# attributes['support'] = 0.01 /4 * 4.5
inputData['fluidGravity'] = inputData['fluidGravity'][:,:2]
fluidPositions, boundaryPositions, fluidFeatures, boundaryFeatures = constructFluidFeatures(attributes, inputData)


predictedPositions = inputData['fluidPosition'].to(device)
predictedVelocity = torch.clone(inputData['fluidVelocity']).to(device)

/home/winchenbach/dev/pytorchSPH2/Cconv/datautils.py:320: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  inFile = h5py.File(filename)


In [10]:
import collections

import learned_simulator
import noise_utils
import numpy as np
import tensorflow.compat.v1 as tf

/home/winchenbach/anaconda3/envs/gns/lib/python3.7/site-packages/tensorflow_core/python/pywrap_tensorflow_internal.py:15: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/winchenbach/anaconda3/envs/gns/lib/python3.7/site-packages/tensorflow_core/python/framework/dtypes.py:597: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/home/winchenbach/anaconda3/envs/gns/lib/python3.7/site-packages/tensorflow_core/python/framework/dtypes.py:605: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically 

In [11]:

INPUT_SEQUENCE_LENGTH = 6
SEQUENCE_LENGTH = INPUT_SEQUENCE_LENGTH + 1  # add one target position.
NUM_DIMENSIONS = 2
NUM_PARTICLE_TYPES = 2
BATCH_SIZE = 2
GLOBAL_CONTEXT_SIZE = 6

Stats = collections.namedtuple("Stats", ["mean", "std"])

DUMMY_STATS = Stats(
    mean=np.zeros([NUM_DIMENSIONS], dtype=np.float32),
    std=np.ones([NUM_DIMENSIONS], dtype=np.float32))
DUMMY_CONTEXT_STATS = Stats(
    mean=np.zeros([GLOBAL_CONTEXT_SIZE], dtype=np.float32),
    std=np.ones([GLOBAL_CONTEXT_SIZE], dtype=np.float32))
DUMMY_BOUNDARIES = [(-1., 1.)] * NUM_DIMENSIONS

In [12]:

def sample_random_position_sequence():
  """Returns mock data mimicking the input features collected by the encoder."""
  num_particles = tf.random_uniform(
      shape=(), minval=50, maxval=1000, dtype=tf.int32)
  position_sequence = tf.random.normal(
      shape=[num_particles, SEQUENCE_LENGTH, NUM_DIMENSIONS])
  return position_sequence


In [13]:

learnable_model = learned_simulator.LearnedSimulator(
  num_dimensions=NUM_DIMENSIONS,
  connectivity_radius=0.05,
  graph_network_kwargs=dict(
      latent_size=128,
      mlp_hidden_size=128,
      mlp_num_hidden_layers=2,
      num_message_passing_steps=2,
  ),
  boundaries=DUMMY_BOUNDARIES,
  normalization_stats={"acceleration": DUMMY_STATS,
                       "velocity": DUMMY_STATS,
                       "context": DUMMY_CONTEXT_STATS,},
  num_particle_types=NUM_PARTICLE_TYPES,
  particle_type_embedding_size=16,
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/home/winchenbach/anaconda3/envs/gns/lib/python3.7/site-packages/sonnet/python/modules/base.py:169: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if isinstance(custom_getter, collections.Mapping):
/home/winchenbach/anaconda3/envs/gns/lib/python3.7/site-packages/sonnet/python/modules/nets/mlp.py:90: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(output_sizes, collections.Iterable):
/home/winchenbach/anaconda3/envs/gns/lib/python3.7/site-packages/tensorflow_core/python/ops/variable_scope.py:452: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  shape is not None and isinstance(shape, collections_lib.Sequence) and
/home/wi

In [14]:

  # Sample a batch of particle sequences with shape:
  # [TOTAL_NUM_PARTICLES, SEQUENCE_LENGTH, NUM_DIMENSIONS]
  sampled_position_sequences = [
      sample_random_position_sequence() for _ in range(BATCH_SIZE)]
  position_sequence_batch = tf.concat(sampled_position_sequences, axis=0)

  # Count how many particles are present in each element in the batch.
  # [BATCH_SIZE]
  n_particles_per_example = tf.stack(
      [tf.shape(seq)[0] for seq in sampled_position_sequences], axis=0)

  # Sample particle types.
  # [TOTAL_NUM_PARTICLES]
  particle_types = tf.random_uniform(
      [tf.shape(position_sequence_batch)[0]],
      0, NUM_PARTICLE_TYPES, dtype=tf.int32)

  # Sample global context.
  global_context = tf.random_uniform(
      [BATCH_SIZE, GLOBAL_CONTEXT_SIZE], -1., 1., dtype=tf.float32)

  # Separate input sequence from target sequence.
  # [TOTAL_NUM_PARTICLES, INPUT_SEQUENCE_LENGTH, NUM_DIMENSIONS]
  input_position_sequence = position_sequence_batch[:, :-1]
  # [TOTAL_NUM_PARTICLES, NUM_DIMENSIONS]
  target_next_position = position_sequence_batch[:, -1]

In [15]:
  # Single step of inference with the model to predict next position for each
  # particle [TOTAL_NUM_PARTICLES, NUM_DIMENSIONS].
  predicted_next_position = learnable_model(
      input_position_sequence, n_particles_per_example, global_context,
      particle_types)
  print(f"Per-particle output tensor: {predicted_next_position}")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Per-particle output tensor: Tensor("LearnedSimulator_1/add_2:0", shape=(?, 2), dtype=float32)


In [16]:

  # Obtaining predicted and target normalized accelerations for training.
  position_sequence_noise = (
      noise_utils.get_random_walk_noise_for_position_sequence(
          input_position_sequence, noise_std_last_step=6.7e-4))

  # Both with shape [TOTAL_NUM_PARTICLES, NUM_DIMENSIONS]
  predicted_normalized_acceleration, target_normalized_acceleration = (
      learnable_model.get_predicted_and_target_normalized_accelerations(
          target_next_position, position_sequence_noise,
          input_position_sequence, n_particles_per_example, global_context,
          particle_types))
  print(f"Predicted norm. acceleration: {predicted_normalized_acceleration}")
  print(f"Target norm. acceleration: {target_normalized_acceleration}")

Predicted norm. acceleration: Tensor("EncodeProcessDecode/mlp/linear_2/add:0", shape=(?, 2), dtype=float32)
Target norm. acceleration: Tensor("truediv_4:0", shape=(?, 2), dtype=float32)


In [17]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
#     print(variable)
#     print(len(shape))
    variable_parameters = 1
    for dim in shape:
#         print(dim)
        variable_parameters *= dim.value
    if variable_parameters > 512:
        print(variable, variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

<tf.Variable 'LearnedSimulator/EncodeProcessDecode/graph_independent/edge_model/mlp/linear_1/w:0' shape=(128, 128) dtype=float32_ref> 16384
<tf.Variable 'LearnedSimulator/EncodeProcessDecode/graph_independent/edge_model/mlp/linear_2/w:0' shape=(128, 128) dtype=float32_ref> 16384
<tf.Variable 'LearnedSimulator/EncodeProcessDecode/graph_independent/node_model/mlp/linear_0/w:0' shape=(36, 128) dtype=float32_ref> 4608
<tf.Variable 'LearnedSimulator/EncodeProcessDecode/graph_independent/node_model/mlp/linear_1/w:0' shape=(128, 128) dtype=float32_ref> 16384
<tf.Variable 'LearnedSimulator/EncodeProcessDecode/graph_independent/node_model/mlp/linear_2/w:0' shape=(128, 128) dtype=float32_ref> 16384
<tf.Variable 'LearnedSimulator/EncodeProcessDecode/interaction_network/edge_block/mlp/linear_0/w:0' shape=(384, 128) dtype=float32_ref> 49152
<tf.Variable 'LearnedSimulator/EncodeProcessDecode/interaction_network/edge_block/mlp/linear_1/w:0' shape=(128, 128) dtype=float32_ref> 16384
<tf.Variable 'Lear

In [42]:
print(len(tf.trainable_variables()))

183


In [37]:

  with tf.train.SingularMonitoredSession() as sess:
    sess.run([predicted_next_position,
              predicted_normalized_acceleration,
              target_normalized_acceleration])


INFO:tensorflow:Graph was finalized.


2023-04-05 12:56:04.978037: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-04-05 12:56:05.005818: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-05 12:56:05.005943: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3060 major: 8 minor: 6 memoryClockRate(GHz): 1.867
pciBusID: 0000:01:00.0
2023-04-05 12:56:05.006126: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2023-04-05 12:56:05.007252: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2023-04-05 12:56:05.008241: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2023-04-05 13:01:11.124603: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2023-04-05 13:02:34.210432: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-05 13:02:34.210457: E tensorflow/stream_executor/cuda/cuda_blas.cc:428] failed to run cuBLAS routine: CUBLAS_STATUS_EXECUTION_FAILED
2023-04-05 13:02:34.210555: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3060 major: 8 minor: 6 memoryClockRate(GHz): 1.867
pciBusID: 0000:01:00.0
2023-04-05 13:02:34.210609: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2023-04-05 13:02:34.210623: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(915, 36), b.shape=(36, 128), m=915, n=128, k=36
	 [[node LearnedSimulator_1/EncodeProcessDecode/graph_independent/node_model/sequential/mlp/linear_0/MatMul (defined at home/winchenbach/anaconda3/envs/gns/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
  (1) Internal: Blas GEMM launch failed : a.shape=(915, 36), b.shape=(36, 128), m=915, n=128, k=36
	 [[node LearnedSimulator_1/EncodeProcessDecode/graph_independent/node_model/sequential/mlp/linear_0/MatMul (defined at home/winchenbach/anaconda3/envs/gns/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
	 [[LearnedSimulator_1/add_2/_759]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'LearnedSimulator_1/EncodeProcessDecode/graph_independent/node_model/sequential/mlp/linear_0/MatMul':
  File "home/winchenbach/anaconda3/envs/gns/lib/python3.7/runpy.py", line 193, in _